In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
                                  max_num_faces=1,
                                  refine_landmarks=True,
                                  min_detection_confidence=0.5,
                                  min_tracking_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils

# Define landmark indices for different facial features
left_eye_indices = [33, 160, 158, 133, 153, 144]
right_eye_indices = [362, 385, 387, 263, 373, 380]
nose_indices = [1, 4, 5, 6, 94, 164, 197, 195, 18]
left_cheek_indices = [205, 50, 101, 118, 202, 281, 426]
right_cheek_indices = [425, 248, 356, 345, 372, 367, 379]
lips_indices = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291]
left_eyebrow_indices = [70, 63, 105, 66, 107]
right_eyebrow_indices = [336, 296, 334, 293, 300]

# Function to draw polygons around facial features
def draw_polygon(frame, landmark_indices, face_landmarks, color):
    points = []
    for idx in landmark_indices:
        x = int(face_landmarks.landmark[idx].x * frame.shape[1])
        y = int(face_landmarks.landmark[idx].y * frame.shape[0])
        points.append((x, y))
    
    points = np.array(points, dtype=np.int32)
    # Draw polygon with the outline
    cv2.polylines(frame, [points], isClosed=True, color=color, thickness=2)

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image to RGB as MediaPipe works with RGB images
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and find face landmarks
    results = face_mesh.process(rgb_frame)

    # If landmarks are found
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Draw polygons around facial features
            draw_polygon(frame, left_eye_indices, face_landmarks, (0, 255, 0))  # Green for left eye
            draw_polygon(frame, right_eye_indices, face_landmarks, (0, 255, 0))  # Green for right eye
            draw_polygon(frame, nose_indices, face_landmarks, (255, 0, 0))  # Blue for nose
            draw_polygon(frame, left_cheek_indices, face_landmarks, (255, 255, 0))  # Yellow for left cheek
            draw_polygon(frame, right_cheek_indices, face_landmarks, (255, 255, 0))  # Yellow for right cheek
            draw_polygon(frame, lips_indices, face_landmarks, (0, 0, 255))  # Red for lips
            draw_polygon(frame, left_eyebrow_indices, face_landmarks, (128, 0, 128))  # Purple for left eyebrow
            draw_polygon(frame, right_eyebrow_indices, face_landmarks, (128, 0, 128))  # Purple for right eyebrow

    # Display the frame with facial feature polygons
    cv2.imshow('Facial Feature Detection (Eyes, Cheeks, Nose, Lips, Eyebrows)', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
